In [154]:
import pandas as pd
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,FunctionTransformer,PowerTransformer,OneHotEncoder

In [155]:
df=pd.read_csv('D:/Competitions/SOI/AI-Clowns/training_datasets/updated.csv')
df.head()
df['time_elapsed'].value_counts

<bound method IndexOpsMixin.value_counts of 0            0
1            1
2            2
3            4
4            5
          ... 
104404    6142
104405    6142
104406    6142
104407    6142
104408    6142
Name: time_elapsed, Length: 104409, dtype: int64>

In [156]:
'''Scaler_pipeline=Pipeline(steps=[
    ('scaler',StandardScaler())
])

log_pipelie=Pipeline(steps=[
    ('log',FunctionTransformer(np.log1p,validate=True)),
    ('scaler',StandardScaler())
])

boxcox_pipeline=Pipeline(steps=[
    ('boxcox',PowerTransformer(method='box-cox')),
    ('scaler',StandardScaler())
])

yeojohnson_pipeline = Pipeline(steps=[
    ('yeojohnson', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])

preprocessor=ColumnTransformer(transformers=[
    ('yeojohnso',yeojohnson_pipeline,['zon.winds']),
    ('log',log_pipelie,['air temp.']),
    ('scaler',Scaler_pipeline,(df.drop(columns=['zon.winds','air temp.','s.s.temp.'])).columns)
])

preprocessed_data_1=preprocessor.fit_transform(df)'''

"Scaler_pipeline=Pipeline(steps=[\n    ('scaler',StandardScaler())\n])\n\nlog_pipelie=Pipeline(steps=[\n    ('log',FunctionTransformer(np.log1p,validate=True)),\n    ('scaler',StandardScaler())\n])\n\nboxcox_pipeline=Pipeline(steps=[\n    ('boxcox',PowerTransformer(method='box-cox')),\n    ('scaler',StandardScaler())\n])\n\nyeojohnson_pipeline = Pipeline(steps=[\n    ('yeojohnson', PowerTransformer(method='yeo-johnson')),\n    ('scaler', StandardScaler())\n])\n\npreprocessor=ColumnTransformer(transformers=[\n    ('yeojohnso',yeojohnson_pipeline,['zon.winds']),\n    ('log',log_pipelie,['air temp.']),\n    ('scaler',Scaler_pipeline,(df.drop(columns=['zon.winds','air temp.','s.s.temp.'])).columns)\n])\n\npreprocessed_data_1=preprocessor.fit_transform(df)"

In [157]:
#Split the data but not randomly, do it in a sequential order

X_train=df[df['time_elapsed']<=5900].drop(columns=['s.s.temp.'])
X_test=df[df['time_elapsed']>5900].drop(columns=['s.s.temp.'])
y_train=df[df['time_elapsed']<=5900]['s.s.temp.']
y_test=df[df['time_elapsed']>5900]['s.s.temp.']


In [158]:
# Transformations and Scaling

yeojohnson_transformer = PowerTransformer(method='yeo-johnson')
X_train['zon.winds'] = yeojohnson_transformer.fit_transform(X_train[['zon.winds']])
X_test['zon.winds'] = yeojohnson_transformer.transform(X_test[['zon.winds']])

'''log_transformer = FunctionTransformer(np.log1p, validate=True)
X_train['air temp.'] = log_transformer.fit_transform(X_train[['air temp.']])
X_test['air temp.'] = log_transformer.transform(X_test[['air temp.']])'''
X_train['air temp.']=yeojohnson_transformer.fit_transform(X_train[['air temp.']])
X_test['air temp.']=yeojohnson_transformer.transform(X_test[['air temp.']])


X_train.drop(columns=['day','time_elapsed'],inplace=True)
X_test.drop(columns=['day','time_elapsed',],inplace=True)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [159]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


# Define parameters for hyperparameter tuning
xgb_param_grid = {'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7], 'n_estimators': [370, 400, 430]}


# Perform hyperparameter tuning for XGBoost
xgb_model = XGBRegressor()
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)
xgb_best_params = xgb_grid_search.best_params_



# Evaluate models
# Assuming X_test and y_test are available
xgb_best_model = XGBRegressor(**xgb_best_params)
xgb_best_model.fit(X_train, y_train)
y_pred_xgb = xgb_best_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)



print("XGBoost MSE:", mse_xgb)

print("XGBoost Best Parameters:", xgb_best_params)



Fitting 5 folds for each of 27 candidates, totalling 135 fits
XGBoost MSE: 0.21267189777471093
XGBoost Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 400}


In [160]:
from sklearn.metrics import r2_score
r_squared_xgb = r2_score(y_test, y_pred_xgb)
print("R² Score xgb:", r_squared_xgb)


R² Score xgb: 0.9632070922283311


In [161]:
comparison = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred_xgb})
comparison.head(15)

,y_test,y_pred
93048,27.77,28.079226
93049,29.25,29.154629
93050,27.52,27.801105
93051,29.89,30.085695
93052,27.80,27.757889
93053,29.80,29.906174
93054,28.69,28.212770
93055,29.27,29.770378
93056,26.73,26.701807
93057,28.90,29.277826


Our first model

In [163]:
import pickle
with open('xgb_best_model.pkl', 'wb') as f:
    pickle.dump(xgb_best_model, f)

print("Model trained and saved successfully using pickle.")

Model trained and saved successfully using pickle.
